

# training con misure

In [2]:

import pandas as pd

from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from joblib import load

import sys

import mlflow
import mlflow.sklearn
import mlflow.keras
from pymongo import MongoClient
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, './../')
import analysis_functions.XSensAnalysis as xaf



default_logged_model = 'runs:/17a2da7e7d3348319edb0a9a669927ee/modello_keras'

default_logged_model_pca = 'runs:/17a2da7e7d3348319edb0a9a669927ee/modello_pca'
default_mongodb_uri = 'mongodb://10.250.4.35:27017'



In [3]:

mlflow_tracking_uri = 'http://10.250.4.35:8000'
mlflow_experiment = 'XSense Training'
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment(mlflow_experiment)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1710847482067, experiment_id='1', last_update_time=1710847482067, lifecycle_stage='active', name='XSense Training', tags={}>

In [4]:
def readjson(first):
    df = {}
    df['Segment Orientation - Quat_Frame'] = first['frameNumber']
    df['Segment Orientation - Quat_Pelvis q0'] = first['segments']["0"]["orientation"]["q1"]
    df['Segment Orientation - Quat_Pelvis q1'] = first['segments']["0"]["orientation"]["q2"]
    df['Segment Orientation - Quat_Pelvis q2'] = first['segments']["0"]["orientation"]["q3"]
    df['Segment Orientation - Quat_Pelvis q3'] = first['segments']["0"]["orientation"]["q4"]
    df['Segment Position_Pelvis x'] = first['segments']["0"]["position"]["x"]
    df['Segment Position_Pelvis y'] = first['segments']["0"]["position"]["y"]
    df['Segment Position_Pelvis z'] = first['segments']["0"]["position"]["z"]
    df['Segment Position_L5 x'] = first['segments']["1"]["position"]["x"]
    df['Segment Position_L5 y'] = first['segments']["1"]["position"]["y"]
    df['Segment Position_L5 z'] = first['segments']["1"]["position"]["z"]
    df['Segment Position_L3 x'] = first['segments']["2"]["position"]["x"]
    df['Segment Position_L3 y'] = first['segments']["2"]["position"]["y"]
    df['Segment Position_L3 z'] = first['segments']["2"]["position"]["z"]
    df['Segment Position_T12 x'] = first['segments']["3"]["position"]["x"]
    df['Segment Position_T12 y'] = first['segments']["3"]["position"]["y"]
    df['Segment Position_T12 z'] = first['segments']["3"]["position"]["z"]
    df['Segment Position_T8 x'] = first['segments']["4"]["position"]["x"]
    df['Segment Position_T8 y'] = first['segments']["4"]["position"]["y"]
    df['Segment Position_T8 z'] = first['segments']["4"]["position"]["z"]
    df['Segment Position_Neck x'] = first['segments']["5"]["position"]["x"]
    df['Segment Position_Neck y'] = first['segments']["5"]["position"]["y"]
    df['Segment Position_Neck z'] = first['segments']["5"]["position"]["z"]
    df['Segment Position_Head x'] =     first['segments']["6"]["position"]["x"]
    df['Segment Position_Head y'] = first['segments']["6"]["position"]["y"]
    df['Segment Position_Head z'] = first['segments']["6"]["position"]["z"]
    df['Segment Position_Right Shoulder x'] = first['segments']["7"]["position"]["x"]
    df['Segment Position_Right Shoulder y'] = first['segments']["7"]["position"]["y"]
    df['Segment Position_Right Shoulder z'] = first['segments']["7"]["position"]["z"]
    df['Segment Position_Right Upper Arm x'] = first['segments']["8"]["position"]["x"]
    df['Segment Position_Right Upper Arm y'] = first['segments']["8"]["position"]["y"]
    df['Segment Position_Right Upper Arm z'] = first['segments']["8"]["position"]["z"]
    df['Segment Position_Right Forearm x'] = first['segments']["9"]["position"]["x"]
    df['Segment Position_Right Forearm y'] = first['segments']["9"]["position"]["y"]
    df['Segment Position_Right Forearm z'] = first['segments']["9"]["position"]["z"]
    df['Segment Position_Left Shoulder x'] = first['segments']["11"]["position"]["x"]
    df['Segment Position_Left Shoulder y'] = first['segments']["11"]["position"]["y"]
    df['Segment Position_Left Shoulder z'] = first['segments']["11"]["position"]["z"]
    df['Segment Position_Left Upper Arm x'] = first['segments']["12"]["position"]["x"]
    df['Segment Position_Left Upper Arm y'] = first['segments']["12"]["position"]["y"]  
    df['Segment Position_Left Upper Arm z'] = first['segments']["12"]["position"]["z"]
    df['Segment Position_Left Forearm x'] = first['segments']["13"]["position"]["x"]
    df['Segment Position_Left Forearm y'] = first['segments']["13"]["position"]["y"]
    df['Segment Position_Left Forearm z'] = first['segments']["13"]["position"]["z"]

    df['Joint Angles ZXY_L5S1 Axial Bending'] = first['joints']["0-1"]["y"]
    df['Joint Angles ZXY_L5S1 Flexion/Extension'] = first['joints']["0-1"]["z"]
    df['Joint Angles ZXY_L4L3 Axial Rotation'] = first['joints']["1-2"]["y"]
    df['Joint Angles ZXY_L4L3 Flexion/Extension'] = first['joints']["1-2"]["z"]
    df['Joint Angles ZXY_L1T12 Axial Rotation'] = first['joints']["2-3"]["y"]
    df['Joint Angles ZXY_L1T12 Flexion/Extension'] = first['joints']["2-3"]["z"]
    df['Joint Angles ZXY_T9T8 Axial Rotation'] = first['joints']["3-4"]["y"]
    df['Joint Angles ZXY_T9T8 Flexion/Extension'] = first['joints']["3-4"]["z"]
    df['Joint Angles ZXY_T1C7 Lateral Bending'] = first['joints']["4-5"]["x"]
    df['Joint Angles ZXY_C1 Head Lateral Bending'] = first['joints']["5-6"]["x"]
    df['Joint Angles ZXY_Right Shoulder Internal/External Rotation'] = first['joints']["7-8"]["y"]
    df['Joint Angles ZXY_Right Shoulder Flexion/Extension'] = first['joints']["7-8"]["z"]
    df['Joint Angles ZXY_Right Elbow Ulnar Deviation/Radial Deviation'] = first['joints']["8-9"]["x"]
    df['Joint Angles ZXY_Right Elbow Pronation/Supination'] = first['joints']["8-9"]["y"]
    df['Joint Angles ZXY_Right Elbow Flexion/Extension'] = first['joints']["8-9"]["z"]
    df['Joint Angles ZXY_Left T4 Shoulder Abduction/Adduction'] = first['joints']["4-11"]["x"]
    df['Joint Angles ZXY_Left T4 Shoulder Internal/External Rotation'] = first['joints']["4-11"]["y"]
    df['Joint Angles ZXY_Left T4 Shoulder Flexion/Extension'] = first['joints']["4-11"]["z"]
    df['Joint Angles ZXY_Left Shoulder Abduction/Adduction'] = first['joints']["11-12"]["x"]
    df['Joint Angles ZXY_Left Shoulder Internal/External Rotation'] = first['joints']["11-12"]["y"]
    df['Joint Angles ZXY_Left Shoulder Flexion/Extension'] = first['joints']["11-12"]["z"]
    df['Joint Angles ZXY_Left Elbow Pronation/Supination'] = first['joints']["12-13"]["y"]
    df['Joint Angles ZXY_Left Elbow Flexion/Extension'] = first['joints']["12-13"]["z"]
    
    return df

def unioneFiles(elena, final):
    # Cicla su tutte le chiavi del dizionario originale
    for chiave, valore in elena.items():
        # Controlla se la parola 'angels' è contenuta nella chiave
        if 'Angles' in chiave:
            # Aggiunge la chiave e il valore al dizionario risultante
            final[chiave] = valore
    colonnePassate = ['Segment Position_Head_y','Segment Position_Head_z','Segment Position_Right Shoulder_x','Segment Position_Right Shoulder_y','Segment Position_Right Shoulder_z','Segment Position_Right Upper Arm_y','Segment Position_Right Upper Arm_z','Segment Position_Right Forearm_y','Segment Position_Left Shoulder_y','Segment Position_Left Shoulder_z','Segment Position_Left Upper Arm_y','Segment Position_Left Forearm_y','Segment Position_Left Shoulder_x','Segment Position_Left Upper Arm_x','Segment Position_Head_x','Joint Angles ZXY_Left Shoulder Abduction/Adduction','Joint Angles ZXY_T9T8 Axial Rotation','Joint Angles ZXY_L1T12 Axial Rotation','Joint Angles ZXY_L4L3 Axial Rotation','Joint Angles ZXY_L5S1 Axial Bending','Joint Angles ZXY_T1C7 Lateral Bending','Joint Angles ZXY_C1 Head Lateral Bending','Joint Angles ZXY_Left T4 Shoulder Flexion/Extension','Segment Position_Right Upper Arm_x','Segment Position_L3_x','Segment Position_L3_y','Segment Position_L3_z','Segment Position_L5_x','Segment Position_L5_y','Segment Position_L5_z','Segment Position_Neck_x','Segment Position_Neck_y','Segment Position_Neck_z','Segment Position_T12_x','Segment Position_T12_y','Segment Position_T12_z','Segment Position_T8_x','Segment Position_T8_y','Segment Position_T8_z']

    # Rimuovi le colonne originali dal dizionario
    for col in colonnePassate:
        try:
            final.pop(col)
        except:
            pass
    return final







In [5]:
mGClient = MongoClient(f'mongodb://10.250.4.35:27017')
# collection = mGClient['experiments']['Exp_prova_christian_good_xsens']

In [6]:

frame_number = []
dfs = pd.DataFrame()    
datasetSenzaTagli = []
desired_order = ['Joint Angles ZXY_L5S1 Flexion/Extension',
'Joint Angles ZXY_L4L3 Flexion/Extension',
'Joint Angles ZXY_L1T12 Flexion/Extension',
'Joint Angles ZXY_T9T8 Flexion/Extension',
'Joint Angles ZXY_Right Shoulder Internal/External Rotation',
'Joint Angles ZXY_Right Shoulder Flexion/Extension',
'Joint Angles ZXY_Right Elbow Ulnar Deviation/Radial Deviation',
'Joint Angles ZXY_Right Elbow Pronation/Supination',
'Joint Angles ZXY_Right Elbow Flexion/Extension',
'Joint Angles ZXY_Left T4 Shoulder Abduction/Adduction',
'Joint Angles ZXY_Left T4 Shoulder Internal/External Rotation',
'Joint Angles ZXY_Left Shoulder Internal/External Rotation',
'Joint Angles ZXY_Left Shoulder Flexion/Extension',
'Joint Angles ZXY_Left Elbow Pronation/Supination',
'Joint Angles ZXY_Left Elbow Flexion/Extension',
'Segment Position_Right Forearm_x',
'Segment Position_Right Forearm_z',
'Segment Position_Left Upper Arm_z',
'Segment Position_Left Forearm_x',
'Segment Position_Left Forearm_z']


In [7]:
def creazione_training(datas, misure):
    dfs = pd.DataFrame()
    for first in datas:  
        frame_number.append(first['frameNumber'])
        df = readjson(first)
        RMatrix, df = xaf.quaternioni(df)
        df, finaleTers = xaf.ternions(df, RMatrix)
        finaleTers = xaf.xyzSeparazione(finaleTers)
        
        datasetSenzaTagli.append(finaleTers)
        finalissimo = unioneFiles(df, finaleTers)
        finalllll = pd.DataFrame(finalissimo, index=[0])
        dfs = pd.concat([dfs, finalllll]).reset_index(drop=True)
    # su tutte le righe di dfs aggiungi i dati di misure
    for key, value in misure.items():
        dfs[key] =value
        
    return dfs

In [8]:
# Name	bodyHeight	footSize	shoulderHeight	shoulderWidth	elbowSpan
# Elena	1.576	0.283	1.305	0.348	0.781
# Stefano	1.819	0.327	1.517	0.348	0.950
# Daniel	1.737	0.285	1.410	0.373	0.870
# Christian	1.813	0.290	1.525	0.335	0.991
# andrea	1.811	0.304	1.502	0.410	0.875
# omar	1.708	0.308	1.404	0.365	0.835


In [9]:
listaEsperimenti = ["Exp_prova_christian_good_xsens", "Exp_omar_good_oven_xsens", "Exp_omar_acceptable_xsens", "Exp_omar_acceptable_oven_xsens", "Exp_andrea_acceptable_oven_xsens", "Exp_andrea_good_oven_xsens", "Exp_andrea_acceptable_xsens"]
misure = {"christian": {"bodyHeight": 1.813, "footSize": 0.290, "shoulderHeight": 1.525, "shoulderWidth": 0.335, "elbowSpan": 0.991}, "omar": {"bodyHeight": 1.708, "footSize": 0.308, "shoulderHeight": 1.404, "shoulderWidth": 0.365, "elbowSpan": 0.835}, "andrea": {"bodyHeight": 1.811, "footSize": 0.304, "shoulderHeight": 1.502, "shoulderWidth": 0.410, "elbowSpan": 0.875}}
personeCompletes = pd.DataFrame()
for exp in listaEsperimenti:
    print(exp)
    
    collection = mGClient['experiments'][exp]
    datas = collection.find({})
    for x in misure:
        if x in exp:
            persona = x
    personaCompleta = creazione_training(datas, misure[persona])
    personeCompletes = pd.concat([personeCompletes, personaCompleta]).reset_index(drop=True)

Exp_prova_christian_good_xsens
Exp_omar_good_oven_xsens
Exp_omar_acceptable_xsens
Exp_omar_acceptable_oven_xsens
Exp_andrea_acceptable_oven_xsens
Exp_andrea_good_oven_xsens
Exp_andrea_acceptable_xsens


In [10]:
personeCompletes["Target"] = 0

In [11]:
personeCompletes

,Segment Position_Right Forearm_x,Segment Position_Right Forearm_z,Segment Position_Left Upper Arm_z,Segment Position_Left Forearm_x,Segment Position_Left Forearm_z,Joint Angles ZXY_L5S1 Flexion/Extension,Joint Angles ZXY_L4L3 Flexion/Extension,Joint Angles ZXY_L1T12 Flexion/Extension,Joint Angles ZXY_T9T8 Flexion/Extension,Joint Angles ZXY_Right Shoulder Internal/External Rotation,...,Joint Angles ZXY_Left Shoulder Internal/External Rotation,Joint Angles ZXY_Left Shoulder Flexion/Extension,Joint Angles ZXY_Left Elbow Pronation/Supination,Joint Angles ZXY_Left Elbow Flexion/Extension,bodyHeight,footSize,shoulderHeight,shoulderWidth,elbowSpan,Target
0,-0.166826,1.318686,1.639883,-0.175670,1.326974,-2.365574,-6.880296,-1.059865,5.041602,0.311424,...,-8.563770,1.233429,103.718892,1.495535,1.813,0.290,1.525,0.335,0.991,0
1,-0.166180,1.318860,1.639850,-0.175818,1.326660,-2.384615,-6.888643,-1.068211,5.035347,0.516259,...,-8.868417,1.085219,103.041295,1.532487,1.813,0.290,1.525,0.335,0.991,0
2,-0.165450,1.319092,1.639861,-0.175908,1.326408,-2.403689,-6.897005,-1.076574,5.029080,0.715500,...,-9.114902,0.951545,102.273216,1.580278,1.813,0.290,1.525,0.335,0.991,0
3,-0.164615,1.319287,1.639889,-0.175847,1.326199,-2.440368,-6.913173,-1.092742,5.016963,0.760316,...,-9.318479,0.766643,101.469351,1.617881,1.813,0.290,1.525,0.335,0.991,0
4,-0.163679,1.319493,1.639946,-0.175454,1.326058,-2.471542,-6.926892,-1.106461,5.006689,0.679531,...,-9.525775,0.594858,100.691854,1.632719,1.813,0.290,1.525,0.335,0.991,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,0.041397,1.357705,1.646103,0.055728,1.349287,0.869391,-5.441651,0.378781,6.119790,25.932381,...,48.340655,13.199721,100.477216,4.325638,1.811,0.304,1.502,0.410,0.875,0
95996,0.041388,1.357678,1.646116,0.055748,1.349280,0.877083,-5.438208,0.382225,6.122374,25.959170,...,48.335339,13.202855,100.468714,4.330270,1.811,0.304,1.502,0.410,0.875,0
95997,0.041412,1.357655,1.646129,0.055816,1.349268,0.885872,-5.434284,0.386149,6.125316,25.969702,...,48.333175,13.207591,100.456037,4.337021,1.811,0.304,1.502,0.410,0.875,0
95998,0.041375,1.357631,1.646143,0.055824,1.349255,0.893353,-5.430939,0.389494,6.127825,25.967264,...,48.332754,13.211104,100.444298,4.345431,1.811,0.304,1.502,0.410,0.875,0


In [12]:
personeCompletes.loc[0+3000:3000+6000, 'Target'] = 1    #2minuti

personeCompletes.loc[12000+3000:15000+6000, 'Target'] = 1   #2minuti
personeCompletes.loc[24000+3000:27000+6000, 'Target'] = 1   #2minuti
personeCompletes.loc[36000+3000:39000+12000, 'Target'] = 1    #3minuti
personeCompletes.loc[54000+3000:57000+12000, 'Target'] = 1   #3minuti
personeCompletes.loc[72000+3000:75000+6000, 'Target'] = 1   #2minuti
personeCompletes.loc[84000+3000:87000+6000, 'Target'] = 1  #2minuti

In [13]:
personeCompletes.shape

(96000, 26)

In [14]:
personeCompletes.columns

Index(['Segment Position_Right Forearm_x', 'Segment Position_Right Forearm_z',
       'Segment Position_Left Upper Arm_z', 'Segment Position_Left Forearm_x',
       'Segment Position_Left Forearm_z',
       'Joint Angles ZXY_L5S1 Flexion/Extension',
       'Joint Angles ZXY_L4L3 Flexion/Extension',
       'Joint Angles ZXY_L1T12 Flexion/Extension',
       'Joint Angles ZXY_T9T8 Flexion/Extension',
       'Joint Angles ZXY_Right Shoulder Internal/External Rotation',
       'Joint Angles ZXY_Right Shoulder Flexion/Extension',
       'Joint Angles ZXY_Right Elbow Ulnar Deviation/Radial Deviation',
       'Joint Angles ZXY_Right Elbow Pronation/Supination',
       'Joint Angles ZXY_Right Elbow Flexion/Extension',
       'Joint Angles ZXY_Left T4 Shoulder Abduction/Adduction',
       'Joint Angles ZXY_Left T4 Shoulder Internal/External Rotation',
       'Joint Angles ZXY_Left Shoulder Internal/External Rotation',
       'Joint Angles ZXY_Left Shoulder Flexion/Extension',
       'Joint Angle

In [15]:

pca = PCA()
testa = personeCompletes['Target']
traina = personeCompletes.drop(['Target'], axis=1)
traina = traina.to_numpy()
pca.fit(traina)
dataset = pca.transform(traina)[:, :2]
# Dividi il dataset in features (X) e target (y)
X_train = dataset  # Assume che la colonna del target si chiami 'Target'
y_train = testa

In [16]:

# Creazione del modello Keras
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu')) 
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1, mode='min')]
# Compilazione del modello
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

w:\lavoro\projects\xsens_analisi\.venv\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
# Addestramento del modello
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, shuffle=True, callbacks=callback, steps_per_epoch=1000)


Epoch 1/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9359 - loss: 0.2359 - val_accuracy: 0.9508 - val_loss: 0.2188
Epoch 2/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9591 - loss: 0.1530 - val_accuracy: 0.9709 - val_loss: 0.2757
Epoch 3/50
 394/1000 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9642 - loss: 0.1346

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9635 - loss: 0.1325 - val_accuracy: 0.9701 - val_loss: 0.2152
Epoch 4/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9644 - loss: 0.1344 - val_accuracy: 0.9725 - val_loss: 0.2370
Epoch 5/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9654 - loss: 0.1246 - val_accuracy: 0.9725 - val_loss: 0.2427
Epoch 6/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9636 - loss: 0.1195 - val_accuracy: 0.9693 - val_loss: 0.2512
Epoch 7/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9653 - loss: 0.1207 - val_accuracy: 0.9690 - val_loss: 0.2488
Epoch 8/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9644 - loss: 0.1283 - val_accuracy: 0.9708 - val_loss: 0.2689
Epoch 9/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9622 - loss: 0.1276 - val_accuracy: 0.9728 - val_loss: 0.2158
Epoch 10/50
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9629 - loss: 0.1285 - val_accura

In [18]:
# salvo il modello su mlflow,
# aggiungo i parametri del modello su processing.py
# parlo con dani riguardo un terzo programma per finetuning da mettere su docker
# tramite mlflow forse questo sarebbe perfetto per fare il finetuning

In [19]:
#salva pca e modello in locale .keras
import joblib
pca_filename = '../modelli/modelli_corpo/pca.pkl'
joblib.dump(pca, pca_filename)
model.save('../modelli/modelli_corpo/modello.keras')

In [20]:
with mlflow.start_run() as run:
     # Log degli iperparametri
     mlflow.log_param("epochs", 1000)
     mlflow.log_param("batch_size", 32)
     mlflow.log_param("validation_split", 0.2)
     
     mlflow.sklearn.log_model(pca, "modello_pca")
     # Log delle metriche del modello
     mlflow.keras.log_model(model, "modello_keras")

2024/06/05 15:54:18 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
w:\lavoro\projects\xsens_analisi\.venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2024/06/05 15:54:24 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


# fineTuning


In [2]:
import numpy as np
from joblib import load

from tensorflow.keras.models import load_model
import paho.mqtt.client as mqtt
import json
import sys
from pymongo.mongo_client import MongoClient
import time

import argparse
#datetime
from datetime import datetime
import mlflow
import mlflow.sklearn
import mlflow.keras
from pymongo import MongoClient
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../')
import analysis_functions.btsAnalysis as baf
import analysis_functions.XSensAnalysis as xaf

import pandas as pd

In [4]:
default_logged_model = 'runs:/a962f0aceae543919833fe9e3fe971ed/modello_keras'

default_logged_model_pca = 'runs:/a962f0aceae543919833fe9e3fe971ed/modello_pca'
default_mongodb_uri = 'mongodb://10.250.4.35:27017'
default_mlflow_tracking_uri = 'http://10.250.4.35:8000'
default_mlflow_experiment = 'XSense Training'
#  '"10.250.4.35", 1883

In [7]:
# Connessione a MongoDB
Mongoclient = MongoClient(default_mongodb_uri)

# Impostazione di MLFlow
mlflow.set_tracking_uri(default_mlflow_tracking_uri)
mlflow.set_experiment(default_mlflow_experiment)



# PCA = mlflow.sklearn.load_model(default_logged_model_pca)
# MODEL = mlflow.pyfunc.load_model(default_logged_model)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1710847482067, experiment_id='1', last_update_time=1710847482067, lifecycle_stage='active', name='XSense Training', tags={}>

In [15]:
upload_time = mlflow.search_runs(experiment_ids=mlflow.get_experiment_by_name(default_mlflow_experiment).experiment_id)["run_id"][0]
#upload time 1710847482067 to format "2024-05-17T16:04:12.630Z"
# upload_time = datetime.fromtimestamp(upload_time / 1000).strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [17]:
upload_time["run_id"][0]

'f39459082aba418d8dfda37d8e19fbfc'

In [25]:
collectio = Mongoclient['experiments']['metadata']

In [26]:
dati = collectio.find({})

In [27]:
dati = list(dati)

In [28]:
from datetime import datetime
upload_time = '2024-04-17T16:04:12.630Z'
# Assumi che 'upload_time' sia una stringa in formato ISO, convertila in un oggetto datetime
upload_time = datetime.fromisoformat(upload_time.replace('Z', '+00:00'))

lista_EXP = []
for x in dati:
    x = dict(x)
    experiment_start_str = x["experimentStart"]
    # Converti la stringa di inizio esperimento in un oggetto datetime
    experiment_start = datetime.fromisoformat(experiment_start_str.replace('Z', '+00:00'))
    
    print(experiment_start)
    if experiment_start > upload_time:
        # uppi = (x["_id"].binary.hex(), x["xsensExperimentData"]["bodyDimensions"])
        uppi = (x["experimentLabel"], x["xsensExperimentData"]["bodyDimensions"])  
        lista_EXP.append(uppi)

# Stampa la lista risultante
print(lista_EXP)


2024-05-17 16:04:12.630000+00:00
2024-05-17 16:09:30.240000+00:00
2024-05-30 10:51:53.563000+00:00
2024-05-30 10:55:28.681000+00:00
2024-05-30 11:01:39.514000+00:00
2024-05-30 11:09:48.611000+00:00
2024-05-30 11:12:30.565000+00:00
2024-05-30 11:17:46.700000+00:00
[('prova_christian1', {'bodyHeight': 1.819, 'footSize': 0.327, 'shoulderHeight': 1.517, 'shoulderWidth': 0.348, 'elbowSpan': 0.95, 'wristSpan': 1.43, 'armSpan': 1.82, 'hipHeight': 1.08, 'hipWidth': 0.401, 'kneeHeight': 0.486125, 'ankleHeight': 0.08, 'shoeSoleHeight': 0.0}), ('prova_christian_good', {'bodyHeight': 1.819, 'footSize': 0.327, 'shoulderHeight': 1.517, 'shoulderWidth': 0.348, 'elbowSpan': 0.95, 'wristSpan': 1.43, 'armSpan': 1.82, 'hipHeight': 1.08, 'hipWidth': 0.401, 'kneeHeight': 0.486125, 'ankleHeight': 0.08, 'shoeSoleHeight': 0.0}), ('andrea_acceptable', {'bodyHeight': 1.819, 'footSize': 0.327, 'shoulderHeight': 1.517, 'shoulderWidth': 0.348, 'elbowSpan': 0.95, 'wristSpan': 1.43, 'armSpan': 1.82, 'hipHeight': 1.0

In [29]:
def creazione_training(datas, misure):
    dfs = pd.DataFrame()
    for first in datas:  
        # frame_number.append(first['frameNumber'])
        df = readjson(first)
        RMatrix, df = xaf.quaternioni(df)
        df, finaleTers = xaf.ternions(df, RMatrix)
        finaleTers = xaf.xyzSeparazione(finaleTers)
        
        datasetSenzaTagli.append(finaleTers)
        finalissimo = unioneFiles(df, finaleTers)
        finalllll = pd.DataFrame(finalissimo, index=[0])
        dfs = pd.concat([dfs, finalllll]).reset_index(drop=True)
    # su tutte le righe di dfs aggiungi i dati di misure
    for key, value in misure.items():
        dfs[key] =value
        
    return dfs

In [30]:
personeCompletes = pd.DataFrame()
for x in lista_EXP:
    misure = {}
    collection = Mongoclient['experiments'][f'Exp_{x[0]}_xsens']
    datas = collection.find({})
    
    misure['bodyHeight'] = x[1]['bodyHeight']
    misure['footSize'] = x[1]['footSize']
    misure['shoulderHeight'] = x[1]['shoulderHeight']
    misure['shoulderWidth'] = x[1]['shoulderWidth']
    misure['elbowSpan'] = x[1]['elbowSpan']


    personaCompleta = creazione_training(datas, misure)
    personeCompletes = pd.concat([personeCompletes, personaCompleta]).reset_index(drop=True)

In [31]:
personeCompletes

,Segment Position_Right Forearm_x,Segment Position_Right Forearm_z,Segment Position_Left Upper Arm_z,Segment Position_Left Forearm_x,Segment Position_Left Forearm_z,Joint Angles ZXY_L5S1 Flexion/Extension,Joint Angles ZXY_L4L3 Flexion/Extension,Joint Angles ZXY_L1T12 Flexion/Extension,Joint Angles ZXY_T9T8 Flexion/Extension,Joint Angles ZXY_Right Shoulder Internal/External Rotation,...,Joint Angles ZXY_Left T4 Shoulder Internal/External Rotation,Joint Angles ZXY_Left Shoulder Internal/External Rotation,Joint Angles ZXY_Left Shoulder Flexion/Extension,Joint Angles ZXY_Left Elbow Pronation/Supination,Joint Angles ZXY_Left Elbow Flexion/Extension,bodyHeight,footSize,shoulderHeight,shoulderWidth,elbowSpan
0,-0.074574,1.331639,1.642446,-0.155683,1.330070,-2.102609,-6.763118,-0.942677,5.121378,6.259744,...,4.370849,0.934347,3.593498,117.409888,0.956250,1.819,0.327,1.517,0.348,0.95
1,-0.074251,1.331593,1.642520,-0.156029,1.330028,-2.087037,-6.756328,-0.935888,5.126473,6.255498,...,4.355416,0.879352,3.468781,117.363587,0.724219,1.819,0.327,1.517,0.348,0.95
2,-0.074059,1.331533,1.642589,-0.156377,1.329988,-2.074198,-6.750799,-0.930359,5.130622,6.244360,...,4.352484,0.816082,3.346130,117.326486,0.493768,1.819,0.327,1.517,0.348,0.95
3,-0.073967,1.331453,1.642634,-0.156767,1.329927,-2.063125,-6.746152,-0.925712,5.134116,6.218460,...,4.333559,0.761580,3.220975,117.288482,0.266847,1.819,0.327,1.517,0.348,0.95
4,-0.073907,1.331354,1.642690,-0.157148,1.329884,-2.045571,-6.738711,-0.918271,5.139709,6.190899,...,4.305680,0.704413,3.095810,117.252404,0.046807,1.819,0.327,1.517,0.348,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,-0.147346,1.330371,1.639569,-0.129858,1.331117,-3.543535,-7.401203,-1.580771,4.651113,4.190734,...,5.456492,6.117820,5.967496,98.737605,3.118309,1.819,0.327,1.517,0.348,0.95
89996,-0.147391,1.330354,1.639583,-0.129807,1.331135,-3.544940,-7.401826,-1.581394,4.650646,4.172481,...,5.458049,6.120700,5.968672,98.739059,3.120771,1.819,0.327,1.517,0.348,0.95
89997,-0.147469,1.330331,1.639590,-0.129779,1.331150,-3.546744,-7.402626,-1.582194,4.650046,4.150197,...,5.461051,6.124076,5.971722,98.737344,3.121070,1.819,0.327,1.517,0.348,0.95
89998,-0.147538,1.330307,1.639597,-0.129752,1.331167,-3.548061,-7.403209,-1.582778,4.649609,4.133644,...,5.461488,6.129127,5.975736,98.735472,3.121261,1.819,0.327,1.517,0.348,0.95


In [32]:
default_logged_model = 'runs:/a962f0aceae543919833fe9e3fe971ed/modello_keras'

default_logged_model_pca = 'runs:/a962f0aceae543919833fe9e3fe971ed/modello_pca'
default_mongodb_uri = 'mongodb://10.250.4.35:27017'
default_mlflow_tracking_uri = 'http://10.250.4.35:8000'
default_mlflow_experiment = 'XSense Training'
#  '"10.250.4.35", 1883


In [33]:
import mlflow
from pymongo import MongoClient
from sklearn.decomposition import PCA
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
import joblib
import numpy as np

In [34]:

mlflow_tracking_uri = 'http://10.250.4.35:8000'
mlflow_experiment = 'XSense Training'
mlflow.set_tracking_uri(mlflow_tracking_uri)
mlflow.set_experiment(mlflow_experiment)
# Funzione per caricare l'ultimo modello da MLflow


# Carica il modello PCA da MLflow
pca_model = mlflow.sklearn.load_model(default_logged_model_pca)

# Carica il modello Keras da MLflow
keras_model = mlflow.keras.load_model(default_logged_model)


In [40]:

# Preprocessa i nuovi dati con il modello PCA
new_data_processed = pca_model.transform(np.array(personeCompletes))[:, :2]

# Effettua predizioni con il modello Keras
predictions = keras_model.predict(new_data_processed)

# Fine-tuning del modello Keras con i nuovi dati
# Assumi di avere le etichette corrette per i nuovi dati in `new_labels`
keras_model.fit(new_data_processed, predictions, epochs=50, batch_size=32, validation_split=0.2)


2813/2813 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
Epoch 1/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1795 - val_accuracy: 0.0000e+00 - val_loss: 0.2039
Epoch 2/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1810 - val_accuracy: 0.0000e+00 - val_loss: 0.2055
Epoch 3/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1783 - val_accuracy: 0.0000e+00 - val_loss: 0.2100
Epoch 4/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1797 - val_accuracy: 0.0000e+00 - val_loss: 0.2066
Epoch 5/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1808 - val_accuracy: 0.0000e+00 - val_loss: 0.2066
Epoch 6/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1802 - val_accuracy: 0.0000e+00 - val_loss: 0.2047
Epoch 7/50
2250/2250 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.0000e+00 - loss: 0.1797 - val_accuracy: 0.0000e+00 - val_loss

In [41]:
#calcola accuracy su training newdata_processed
prediction = keras_model.predict(new_data_processed)
accuracy = np.mean((prediction.astype(int) == predictions.astype(int)).astype(int))
print(f"Accuracy: {accuracy}")

2813/2813 ━━━━━━━━━━━━━━━━━━━━ 3s 904us/step
Accuracy: 1.0
